In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

In [ ]:
import numpy as np
import pandas as pd
# 코사인 유사도 함수
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

# 자카드 유사도 함수
def jaccard_similarity(text1, text2):
    set1, set2 = set(text1.split()), set(text2.split())  # 단어 집합 생성
    intersection = len(set1.intersection(set2))  # 교집합 크기
    union = len(set1.union(set2))  # 합집합 크기
    return intersection / union if union != 0 else 0

# 점수 계산 함수
def calculate_scores(df):
    cosine_scores = []
    jaccard_scores = []

    for i in range(len(df)):
        sentence1 = df["predicted_answer"].iloc[i]
        sentence2 = df["actual_answer"].iloc[i]

        # 문장 임베딩
        embedding1 = model.encode(sentence1, convert_to_numpy=True)
        embedding2 = model.encode(sentence2, convert_to_numpy=True)

        # 유사도 계산
        cosine_sim = cosine_similarity(embedding1, embedding2)
        jaccard_sim = jaccard_similarity(sentence1, sentence2)

        # 리스트에 저장
        cosine_scores.append(cosine_sim)
        jaccard_scores.append(jaccard_sim)

    # 새로운 컬럼 추가
    df["cosine_similarity"] = cosine_scores
    df["jaccard_similarity"] = jaccard_scores

    # 전체 평균 계산
    avg_cosine = np.mean(cosine_scores)
    avg_jaccard = np.mean(jaccard_scores)
    result = avg_cosine*0.7 + avg_jaccard*0.3

    print(f"전체 평균 코사인 유사도: {avg_cosine:.4f}")
    print(f"전체 평균 자카드 유사도: {avg_jaccard:.4f}")
    print(f"전체 평균  유사도: {result:.4f}")

    return df


# 데이터프레임에 유사도 추가
#result_df = calculate_scores(result_df)



In [ ]:
import pandas as pd

# CSV 파일 읽기
df1 = pd.read_csv("C:/Users/minkyu/Desktop/test/rule_base_v2.csv")
df2 = pd.read_csv("C:/Users/minkyu/Desktop/test/output.csv")
pattern = r'[^A-Za-z\uAC00-\uD7A3\s]'

# df2의 각 컬럼에 대해 특수문자 제거 및 문자열 변환 적용
df2['재발방지대책'] = df2['재발방지대책'].fillna('').astype(str).str.replace(pattern, '', regex=True)
df2['향후조치계획'] = df2['향후조치계획'].fillna('').astype(str).str.replace(pattern, '', regex=True)

# df1의 비교할 컬럼도 문자열로 변환
df1['재발방지대책 및 향후조치계획'] = df1['재발방지대책 및 향후조치계획'].fillna('').astype(str)

# 첫번째 비교: predicted_answer vs. actual_answer(재발방지대책)
compare_df1 = pd.DataFrame({
    "predicted_answer": df1['재발방지대책 및 향후조치계획'],
    "actual_answer": df2['재발방지대책']
})

# 두번째 비교: predicted_answer vs. actual_answer(향후조치계획)
compare_df2 = pd.DataFrame({
    "predicted_answer": df1['재발방지대책 및 향후조치계획'],
    "actual_answer": df2['향후조치계획']
})

# calculate_scores 함수를 이용해 유사도 계산 (두 DataFrame 모두에 적용)
print("답지 재발방지대책 기준")
compare_df1 = calculate_scores(compare_df1)
print("-"*20)
print("답지 향후조치계획 기준")
compare_df2 = calculate_scores(compare_df2)


In [ ]:
df2.to_csv("output2.csv",encoding='utf-8-sig')

In [ ]:
답지 재발방지대책 기준
전체 평균 코사인 유사도: 0.5409
전체 평균 자카드 유사도: 0.0499
전체 평균  유사도: 0.3936
--------------------
답지 향후조치계획 기준
전체 평균 코사인 유사도: 0.5461
전체 평균 자카드 유사도: 0.0464
전체 평균  유사도: 0.3962